In [9]:
from statsmodels.tsa.api import ExponentialSmoothing, SimpleExpSmoothing, Holt
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib.ticker import FuncFormatter
%config InlineBackend.figure_format='retina'

In [10]:
pd.set_option('display.max_rows',100)

In [17]:
def createpath(predict_date,num_day):
    predict_day=int(predict_date[8:])
    predict_month=int(predict_date[5:7])
    predict_year=int(predict_date[0:4])
    daybefore=[]
    pmb=predict_month-1
    dayleft=num_day-predict_day
    if predict_day>=num_day:
        day=predict_day-num_day
        while day<predict_day:
            daybefore.append(str(predict_year)+'_'+str(predict_month).zfill(2)+'_'+str(day).zfill(2))
            day+=1
    else:
        if pmb==1 or pmb==3 or pmb==5 or pmb==7 or pmb==8 or pmb==10 or pmb==12:
            for i in range(31-dayleft,32,1):
                daybefore.append(str(predict_year)+'_'+str(pmb).zfill(2)+'_'+str(i).zfill(2))
            for i in range(1,predict_day,1):
                daybefore.append(str(predict_year)+'_'+str(predict_month).zfill(2)+'_'+str(i).zfill(2))
        elif pmb==4 or pmb==6 or pmb==9 or pmb==11:
            for i in range(30-dayleft,31,1):
                daybefore.append(str(predict_year)+'_'+str(pmb).zfill(2)+'_'+str(i).zfill(2))
            for i in range(1,predict_day,1):
                daybefore.append(str(predict_year)+'_'+str(predict_month).zfill(2)+'_'+str(i).zfill(2))
        else:
            if predict_year%4==0:
                for i in range(29-dayleft,30,1):
                    daybefore.append(str(predict_year)+'_'+str(pmb).zfill(2)+'_'+str(i).zfill(2))
                for i in range(1,predict_day,1):
                    daybefore.append(str(predict_year)+'_'+str(predict_month).zfill(2)+'_'+str(i).zfill(2))
            else:
                for i in range(28-dayleft,29,1):
                    daybefore.append(str(predict_year)+'_'+str(pmb).zfill(2)+'_'+str(i).zfill(2))
                for i in range(1,predict_day,1):
                    daybefore.append(str(predict_year)+'_'+str(predict_month).zfill(2)+'_'+str(i).zfill(2))
    pathlist=[]
    for i in daybefore:
        pathlist.append('Energy_Balance_'+str(i))
    return pathlist,predict_date
    

In [12]:
def clean(path):
    rawdata=pd.read_csv(path+".csv", delimiter=";" )
    deleteindex=rawdata.loc[rawdata.loc[pd.IndexSlice[:,'Battery discharging / Mean values [W] ']]==' '].index
    rawdata.drop(deleteindex,inplace=True)
    rawdata.rename(columns={' ':'Time'},inplace=True)
    rawdata['Time']=rawdata['Time'].str.replace('[^\d]','')
    rawdata['Time']=path[15:19]+path[20:22]+path[23:]+rawdata['Time']
    rawdata['Time']=pd.to_datetime(rawdata.Time)
    rawdata.set_index('Time', inplace=True)
    for i in rawdata.columns:
        if i != 'Direct consumption / Mean values [W] ' and i != 'Battery discharging / Mean values [W] ' and i!='External energy supply / Mean values [W] ' and i != 'PV power generation / Mean values [W]  ':
            rawdata.drop([i],axis=1,inplace=True)
    for i in rawdata.columns:
        rawdata[i].fillna('0',inplace=True)
        rawdata[i]=rawdata[i].apply(lambda x: x.replace(' ', '0'))
        rawdata[i]=rawdata[i].str.replace('\D','').astype(int)
    rawdata['Consumption']=rawdata['Direct consumption / Mean values [W] ']+rawdata['Battery discharging / Mean values [W] ']+rawdata['External energy supply / Mean values [W] ']
    rawdata.rename(columns={'PV power generation / Mean values [W]  ':'Generation'},inplace=True)
    rawdata.drop(['Direct consumption / Mean values [W] ', 'Battery discharging / Mean values [W] ','External energy supply / Mean values [W] '],axis=1,inplace=True)
    rawdata.drop(['Generation'],axis=1,inplace=True)
    return rawdata

In [13]:
def pulldata(pathlist):
    rawdata=[]
    for i in pathlist:
        df=clean(i)
        rawdata.append(df)
    data=rawdata[0]
    for i in range(num_day-1):
        data=pd.concat([data,rawdata[i-1]])
    return (data)

In [14]:
def completemissing (data):
    for i in range(data.shape[0]):
        if data.iloc[i,0]==0:
            data.iloc[i,0]=data.iloc[i-96,0]
    return data

In [ ]:
def completemissingpredict (compare):
    

In [19]:
def TestTripleExpo (data):
    fit4 = ExponentialSmoothing(data.Consumption, seasonal_periods=96, trend='add' ,seasonal='mul').fit()
    forecast=fit4.forecast(96)
    compare=clean('Energy_Balance_'+predict_date)
    pmb=predict_month-1
    if predict_day>1:
        db=(str(predict_year)+'_'+str(predict_month).zfill(2)+'_'+str(predict_day-1).zfill(2))
    else:
        if pmb==1 or pmb==3 or pmb==5 or pmb==7 or pmb==8 or pmb==10 or pmb==12:
            db=(str(predict_year)+'_'+str(pmb).zfill(2)+'_'+str(31).zfill(2))
        elif pmb==4 or pmb==6 or pmb==9 or pmb==11:
            db=(str(predict_year)+'_'+str(pmb).zfill(2)+'_'+str(30).zfill(2))
        else:
            if predict_year%4==0:
                db=(str(predict_year)+'_'+str(pmb).zfill(2)+'_'+str(29).zfill(2))
            else:
                db=(str(predict_year)+'_'+str(pmb).zfill(2)+'_'+str(38).zfill(2))    
    dayb=clean('Energy_Balance_'+db)
    bf=clean('Energy_Balance_'+str(predict_year)+'_'+str(predict_month).zfill(2)+'_'+str(predict_day-1).zfill(2))
    for i in range(compare.shape[0]):
        if compare.iloc[i,0]==0:
            compare.iloc[i,0]=dayb.iloc[i,0]
    for i in range(96):
        forecastcolumn.append(forecast.iloc[i])
    compare['Forecast']=forecastcolumn
    compare.rename(columns={'Consumption':'Real'},inplace=True)
    compare['Error']=compare['Real']-compare['Forecast']
    compare['absError/Real']=abs(compare['Error'])/compare['Real']
    MAPE=compare['absError/Real'].sum()*100/96
    return (compare,fit4.model.params)